1. Extração
- Nesta etapa, vamos extrair o CSV para a camada mais baixa (bronze) onde os dados brutos serão armazenados. 

In [56]:
# Importando as bibliotecas
import pandas as pd
import requests
import os
from datetime import date, timedelta, datetime
from glob import glob
from zipfile import BadZipFile

In [57]:
def generate_weekly_file():
    today = date.today()
    
    last_week_end = today - timedelta(days=today.weekday() + 1)
    last_week_start = last_week_end - timedelta(days=6)

    url = (
        f"https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/"
        f"precos/arquivos-lpc/{last_week_end.year}/"
        f"resumo_semanal_lpc_{last_week_start:%Y%m%d}_{last_week_end:%Y%m%d}.xlsx"
    )

    return url, last_week_start.strftime("%Y-%m-%d"), last_week_end.strftime("%Y-%m-%d")

In [58]:
def collect_raw_data():
    current_year = date.today().year
    base_path = f"../data/bronze/{current_year}"
    os.makedirs(base_path, exist_ok=True)

    url, week_start, week_end = generate_weekly_file()
    date_end = datetime.strptime(week_end, "%Y-%m-%d").date()

    
    month = date_end.month
    mkdir_month = f"{base_path}/{month:02d}"
    os.makedirs(mkdir_month, exist_ok=True)

    file_path = f"{mkdir_month}/{week_start}_{week_end}.xlsx"

    
    if os.path.exists(file_path):
        print("Não há atualizações novas")
        return

    
    with open(file_path, "wb") as f:
        f.write(requests.get(url).content)
    print(f"Arquivo salvo: {file_path}")
    

In [59]:
collect_raw_data()

Não há atualizações novas


2. Transformação 
- Transformando os dados em CSV (dados consumíveis e consistentes)

In [60]:
def generate_weekly_file_in_csv():
    current_year = date.today().year
    bronze_files = glob(f"../data/bronze/{current_year}/*/*.xlsx")
    silver_path = f"../data/silver/{current_year}"
    os.makedirs(silver_path, exist_ok=True)

    for file in bronze_files:
        try:
            
            df_temp = pd.read_excel(file, engine="openpyxl")
            header_row = df_temp.index[df_temp.iloc[:,0] == "DATA INICIAL"][0]

            
            df = pd.read_excel(file, engine="openpyxl", header=header_row)

            
            month = os.path.basename(os.path.dirname(file))
            month_path = os.path.join(silver_path, month)
            os.makedirs(month_path, exist_ok=True)

            
            base_name = os.path.basename(file).replace(".xlsx", ".csv")
            csv_file = os.path.join(month_path, base_name)

            
            df.to_csv(csv_file, index=False, encoding="utf-8-sig")
            print(f"Arquivo convertido: {csv_file}")

        except (BadZipFile, ValueError) as e:
            print(f"Arquivo inválido (pulado): {file} ({e})")
        except Exception as e:
            print(f"Erro inesperado em {file}: {e}")


In [61]:
generate_weekly_file_in_csv()

Arquivo convertido: ../data/silver/2025/01/2025-01-19_2025-01-25.csv
Arquivo convertido: ../data/silver/2025/01/2025-01-12_2025-01-18.csv
Arquivo convertido: ../data/silver/2025/01/2025-01-05_2025-01-11.csv
Arquivo convertido: ../data/silver/2025/01/2024-12-29_2025-01-04.csv
Arquivo convertido: ../data/silver/2025/04/2025-03-30_2025-04-05.csv
Arquivo convertido: ../data/silver/2025/04/2025-04-13_2025-04-19.csv
Arquivo convertido: ../data/silver/2025/04/2025-04-20_2025-04-26.csv
Arquivo convertido: ../data/silver/2025/04/2025-04-06_2025-04-12.csv
Arquivo convertido: ../data/silver/2025/02/2025-02-09_2025-02-15.csv
Arquivo convertido: ../data/silver/2025/02/2025-02-16_2025-02-22.csv
Arquivo convertido: ../data/silver/2025/02/2025-01-26_2025-02-01.csv
Arquivo convertido: ../data/silver/2025/02/2025-02-02_2025-02-08.csv
Arquivo convertido: ../data/silver/2025/06/2025-06-22_2025-06-28.csv
Arquivo convertido: ../data/silver/2025/06/2025-06-01_2025-06-07.csv
Arquivo convertido: ../data/silver

In [66]:
def cleaning_all_silver():
  current_year = date.today().year
  silver_path = glob(f"../data/silver/{current_year}/*/*.csv")
  
  for file in silver_path:
    try:
      df = pd.read_csv(file, encoding="utf-8-sig")
      df.columns = ["" if col.startswith("Unnamed") else col for col in df.columns]
      df.columns = df.columns.str.strip().str.upper()
      if "DATA INICIAL" in df.columns:
                df["DATA INICIAL"] = pd.to_datetime(df["DATA INICIAL"], errors="coerce").dt.strftime("%d-%m-%y")
      if "DATA FINAL" in df.columns:
                df["DATA FINAL"] = pd.to_datetime(df["DATA FINAL"], errors="coerce").dt.strftime("%d-%m-%y")      
      df.to_csv(file, index=False, encoding="utf-8-sig") 
      print(df.head(1)) 
    except Exception as e:
            print(f"Erro inesperado em {file}: {e}")

In [67]:
cleaning_all_silver()

                                                         \
0  DATA INICIAL  DATA FINAL  ESTADO  MUNICÍPIO  PRODUTO   

                                                                         \
0  NÚMERO DE POSTOS PESQUISADOS  UNIDADE DE MEDIDA  PREÇO MÉDIO REVENDA   

                                                                      \
0  DESVIO PADRÃO REVENDA  PREÇO MÍNIMO REVENDA  PREÇO MÁXIMO REVENDA   

                             
0  COEF DE VARIAÇÃO REVENDA  
                                                         \
0  DATA INICIAL  DATA FINAL  ESTADO  MUNICÍPIO  PRODUTO   

                                                                         \
0  NÚMERO DE POSTOS PESQUISADOS  UNIDADE DE MEDIDA  PREÇO MÉDIO REVENDA   

                                                                      \
0  DESVIO PADRÃO REVENDA  PREÇO MÍNIMO REVENDA  PREÇO MÁXIMO REVENDA   

                             
0  COEF DE VARIAÇÃO REVENDA  
                                                  